# Sentimen Analisis Report Indo

In [ ]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer

### load dataset

In [ ]:
print("reading")
df = pd.read_csv('lib/dataset',index_col=0)
df1 = pd.read_csv('lib/dataset ahok',index_col=0)
df2 = pd.read_csv('lib/datasetfm',index_col=0)
df=pd.concat([df,df1,df2],sort=False)
#df['target']=df['target'].map({0:-1,1:0,2:1})

df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df = shuffle(df)
df.info()


### downsampling

In [ ]:
dneg=df.query('target == -1')
dnet=df.query('target == 0')
dpos=df.query('target == 1')

minlen=min(len(dnet.index),len(dpos.index),len(dneg.index))

dnet=dnet.head(minlen)
dneg=dneg.head(minlen)
dpos=dpos.head(minlen)

data_training=pd.concat([dneg.text,dnet.text,dpos.text])
target=pd.concat([dneg.target,dnet.target,dpos.target])
target=to_categorical(target-target.min())

### tidak downsampling

In [ ]:
#data_training=df.text
#target=to_categorical(target-target.min())

### tokenizing

In [ ]:
tokenizer = Tokenizer(num_words=100000)
with open('lib/tokenizer_indo.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
sequences = tokenizer.texts_to_sequences(data_training) 

### padding

In [ ]:
input_seq = pad_sequences(sequences, maxlen=70)

### data

In [ ]:
input_val=input_seq
target_val=target
print("data : ",len(input_val))

### fungsi load model

In [ ]:
def load_model(ledom):
    json_file = open(ledom, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    return model

### fungsi acc score

In [ ]:
def score(model,we):
    model.load_weights(we)
    scores = model.evaluate(input_val, target_val)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

### fungsi f1 score

In [ ]:
def f1(model):
    predict = model.predict_classes(input_val)
    y_true = np.argmax(target_val, axis=1, out=None)
    y_pred = predict
    target_names = ['negative', 'neutral', 'positive']
    print (classification_report(y_true, y_pred, target_names=target_names))

### fungsi report

In [ ]:
def report(model_name,model_path,weights_path, s,f):
    for x in range(len(model_name)):
        print("\n\n","="*20,model_name[x],"="*20)
        model=load_model(model_path[x])
        if s==True:
            score(model,weights_path[x])
            print()
        if f==True:
            f1(model)

### set nama model, path model, path weights

In [ ]:
model_name=["Word2Vec",
            "DBOW",
            "DMM",
            "DMC",
            "DBOW-DMM",
            "DBOW-DMC",
            "DMM-DMC",
            "DBOW-DMM-DMC"]

model_path=["lib/model_w2v_indo.json",
           "lib/model_dbow_indo.json",
           "lib/model_dmm_indo.json",
           "lib/model_dmc_indo.json",
           "lib/model_dbow-dmm_indo.json",
           "lib/model_dbow-dmc_indo.json",
           "lib/model_dmm-dmc_indo.json",
            "lib/model_dbow-dmm-dmc_indo.json"]

weights_path=['lib/weights_w2v.hdf5',
           "lib/weights_dbow.hdf5",
           "lib/weights_dmm.hdf5",
           "lib/weights_dmc.hdf5",
           "lib/weights_dbow-dmm.hdf5",
           "lib/weights_dbow-dmc.hdf5",
           "lib/weights_dmm-dmc.hdf5",
            "lib/weights_dbow-dmm-dmc.hdf5"]

# report

parameter
1. list nama model
2. list path model
3. list path weight
4. acc score 
5. f1 score

In [ ]:
report(model_name,model_path,weights_path,True,True)